Here's a summary of all the steps we've gone through to analyze and visualize the urban CI nodes accessibility and service areas:

**1 | Load and add layers:**
> Load the point layers representing urban CI nodes.

> Load the line layer representing the road network (RD).

> Load the flood extent layers for 2011, 2030, and 2050.

*The Kernel Density analysis will show the distribution of urban CI nodes based on their service provision radius. This will give you an understanding of the service areas covered by each CI node.*

---

**2 | Calculate service areas:**
> Use QNEAT3's Iso-Area algorithm to calculate the service areas for each urban CI node based on the given service provision radius and the road network.

*The shortest paths calculated between the population centroids and the urban CI nodes on the road network will give you an understanding of the accessibility of urban CI nodes for the population.*


---


**3 | Load the population grid layer (POP) and create centroid points.**


---



**4 | Calculate the shortest paths:**
> Use QNEAT3's "Shortest path (point to layer)" algorithm to calculate the shortest paths from the population centroids to the urban CI nodes on the road network (RD).


**5 | Remove edges located in flood extent layers from the road network.**

*By removing the edges located in the flood extent layers, you'll be able to analyze the impact of flood events on the road network and its connectivity.*

---


**6 | Calculate the shortest paths using the modified road network:**
> Use QNEAT3's "Shortest path (point to layer)" algorithm to calculate the shortest paths from the population centroids to the urban CI nodes on the modified road network (after removing edges located in flood extent).

*By calculating the shortest paths on the modified road network, you can understand how the accessibility of urban CI nodes is affected by flood events.*

---


**7 | Export the shortest path results to CSV files and Visualize the accessibility of urban CI nodes as Iso-Areas:**

> Extract distances from the shortest path layers.

> Generate Iso-Areas (contour polygons) from the distance values.

*Iso-Areas will show the accessibility of urban CI nodes at different distance intervals. By comparing Iso-Areas before and after removing the edges located in flood extents, you can visualize the impact of floods on the accessibility of urban CI nodes.*



---




In [ ]:
from qgis.core import QgsProject, QgsVectorLayer, QgsLayerTreeLayer

def load_and_add_layer(layer_path, layer_name, layer_type='ogr'):
    layer = QgsVectorLayer(layer_path, layer_name, layer_type)
    if layer.isValid():
        QgsProject.instance().addMapLayer(layer)
    else:
        print(f"Failed to load {layer_name}")
    return layer

# Load point layers
urban_ci_layers = {
    "RAIL": "/path/to/rail_stations.shp",
    "ELEC": "/path/to/electric_power_supply.shp",
    "WAT": "/path/to/water_supply.shp",
    "HLT": "/path/to/healthcare_service.shp",
    "EDU": "/path/to/educational_service.shp",
    "CIV": "/path/to/civic_protection.shp",
    "SAN": "/path/to/sanitation.shp",
}

for layer_name, layer_path in urban_ci_layers.items():
    load_and_add_layer(layer_path, layer_name)

# Load road network layer
road_layer = load_and_add_layer("/path/to/road_network.shp", "RD")

# Load flood extent layers
flood_layers = {
    "FD30": "/path/to/flood_extent_2030.shp",
    "FD50": "/path/to/flood_extent_2050.shp",
    "FD11": "/path/to/flood_extent_2011.shp",
}

for layer_name, layer_path in flood_layers.items():
    load_and_add_layer(layer_path, layer_name)

To perform Kernel Density Estimation (KDE) for the urban CI nodes with their respective service provision radius:

In [ ]:
from qgis.analysis import QgsNativeAlgorithms, QgsOverlayAnalyzer

def remove_nodes_in_flood_extent(ci_layer, flood_layer):
    QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
    params = {
        'INPUT': ci_layer,
        'OVERLAY': flood_layer,
        'OUTPUT': 'memory:'
    }
    result = processing.run("native:difference", params)
    return result['OUTPUT']

In [ ]:
from qgis.analysis import QgsKernelDensityEstimation

def perform_kde(input_layer, radius, output_path):
    kde_params = {
        'INPUT': input_layer,
        'RADIUS': radius,
        'PIXEL_SIZE': 1000, 
        'OUTPUT': output_path
    }
    result = processing.run("qgis:kde", kde_params)
    return result['OUTPUT']

In [ ]:
service_radii = {
    "RAIL": 3000,
    "ELEC": None,
    "WAT": None,
    "HLT-HOS": 10000,
    "HLT-HC": 3000,
    "EDU": 3000,
    "CIV": 3000,
    "SAN": 5000
}

In [ ]:
import os

output_dir = "/path/to/output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for layer_name, layer in urban_ci_layers.items():
    if layer_name not in service_radii:
        continue

    radius = service_radii[layer_name]
    if radius is None:
        continue

    # Remove nodes within flood extent layers
    for flood_layer_name, flood_layer in flood_layers.items():
        layer = remove_nodes_in_flood_extent(layer, flood_layer)

    # Perform Kernel Density Estimation
    kde_output_path = os.path.join(output_dir, f"{layer_name}_kde.tif")
    perform_kde(layer, radius, kde_output_path)

    # Load the KDE raster layer to the QGIS map
    kde_raster_layer = QgsRasterLayer(kde_output_path, f"{layer_name} KDE")
    QgsProject.instance().addMapLayer(kde_raster_layer)

To perform Accessibility analysis by QNEAT3

In [ ]:
# Load population grid layer
pop_grid_layer = load_and_add_layer("/path/to/population_grid.shp", "POP")

# Create centroid layer
params = {
    'INPUT': pop_grid_layer,
    'ALL_PARTS': True,
    'OUTPUT': 'memory:'
}
result = processing.run("native:centroids", params)
pop_centroid_layer = result['OUTPUT']

In [ ]:
def calculate_shortest_paths(pop_layer, ci_layer, network_layer, output_path):
    params = {
        'INPUT': network_layer,
        'START_POINT': pop_layer,
        'END_POINT': ci_layer,
        'STRATEGY': 0,  # Use the "Shortest Path (distance optimization)" strategy
        'ENTRY_COST_PREDICATE': 0,  # Use the "Less Than" predicate
        'ENTRY_COST_VALUE': 0,
        'OUTPUT': output_path
    }
    result = processing.run("qneat3:shortestpathpointtolayer", params)
    return result['OUTPUT']

Calculate the shortest paths for each CI node layer:

In [ ]:
import os

output_dir = "/path/to/output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for layer_name, ci_layer in urban_ci_layers.items():
    shortest_paths_output_path = os.path.join(output_dir, f"{layer_name}_shortest_paths.shp")
    calculate_shortest_paths(pop_centroid_layer, ci_layer, road_layer, shortest_paths_output_path)

    # Load the shortest paths layer to the QGIS map
    shortest_paths_layer = QgsVectorLayer(shortest_paths_output_path, f"{layer_name} Shortest Paths", "ogr")
    QgsProject.instance().addMapLayer(shortest_paths_layer)


Remove edges located in the flood extent from the road network:

In [ ]:
def remove_edges_in_flood_extent(network_layer, flood_layer, output_path):
    params = {
        'INPUT': network_layer,
        'OVERLAY': flood_layer,
        'OUTPUT': output_path
    }
    result = processing.run("native:difference", params)
    return result['OUTPUT']

# Remove edges from the road network that are within the flood extents
for flood_layer_name, flood_layer in flood_layers.items():
    road_layer = remove_edges_in_flood_extent(road_layer, flood_layer, f"memory:{flood_layer_name}_filtered")

Calculate the shortest paths for each CI node layer using the modified road network:

In [ ]:
for layer_name, ci_layer in urban_ci_layers.items():
    shortest_paths_output_path = os.path.join(output_dir, f"{layer_name}_shortest_paths_filtered.shp")
    calculate_shortest_paths(pop_centroid_layer, ci_layer, road_layer, shortest_paths_output_path)

    # Load the filtered shortest paths layer to the QGIS map
    shortest_paths_layer = QgsVectorLayer(shortest_paths_output_path, f"{layer_name} Shortest Paths Filtered", "ogr")
    QgsProject.instance().addMapLayer(shortest_paths_layer)

In [ ]:
def export_layer_to_csv(layer, output_path):
    params = {
        'INPUT': layer,
        'OUTPUT': output_path
    }
    processing.run("native:savefeatures", params)

In [ ]:
csv_output_dir = "/path/to/csv_output"
if not os.path.exists(csv_output_dir):
    os.makedirs(csv_output_dir)

for layer_name, ci_layer in urban_ci_layers.items():
    # Export shortest paths
    csv_output_path = os.path.join(csv_output_dir, f"{layer_name}_shortest_paths.csv")
    shortest_paths_layer = QgsProject.instance().mapLayersByName(f"{layer_name} Shortest Paths")[0]
    export_layer_to_csv(shortest_paths_layer, csv_output_path)

    # Export filtered shortest paths
    csv_output_path = os.path.join(csv_output_dir, f"{layer_name}_shortest_paths_filtered.csv")
    shortest_paths_filtered_layer = QgsProject.instance().mapLayersByName(f"{layer_name} Shortest Paths Filtered")[0]
    export_layer_to_csv(shortest_paths_filtered_layer, csv_output_path)

For Iso-Area

In [ ]:
def extract_distances(layer, output_path):
    params = {
        'INPUT': layer,
        'FIELD_NAME': 'distance',
        'OUTPUT': output_path
    }
    result = processing.run("qgis:fieldcalculator", params)
    return result['OUTPUT']

In [ ]:
def create_iso_areas(layer, distance_interval, output_path):
    params = {
        'INPUT': layer,
        'ATTRIBUTE_NAME': 'distance',
        'METHOD': 0,  # Use the "Classes" method
        'NUMBER_OF_CLASSES': 0,
        'RANGES': distance_interval,
        'OUTPUT': output_path
    }
    result = processing.run("native:createcontours", params)
    return result['OUTPUT']

In [ ]:
import os

iso_output_dir = "/path/to/iso_output"
if not os.path.exists(iso_output_dir):
    os.makedirs(iso_output_dir)

distance_interval = 1000  # You can adjust this value to create Iso-Areas at different intervals

for layer_name, ci_layer in urban_ci_layers.items():
    # Process original shortest paths
    shortest_paths_layer = QgsProject.instance().mapLayersByName(f"{layer_name} Shortest Paths")[0]
    distance_layer_path = os.path.join(iso_output_dir, f"{layer_name}_distances.shp")
    distance_layer = extract_distances(shortest_paths_layer, distance_layer_path)

    iso_areas_output_path = os.path.join(iso_output_dir, f"{layer_name}_iso_areas.shp")
    create_iso_areas(distance_layer, distance_interval, iso_areas_output_path)

    # Load the Iso-Areas layer to the QGIS map
    iso_areas_layer = QgsVectorLayer(iso_areas_output_path, f"{layer_name} Iso Areas", "ogr")
    QgsProject.instance().addMapLayer(iso_areas_layer)

    # Process filtered shortest paths
    shortest_paths_filtered_layer = QgsProject.instance().mapLayersByName(f"{layer_name} Shortest Paths Filtered")[0]
    distance_layer_path = os.path.join(iso_output_dir, f"{layer_name}_distances_filtered.shp")
    distance_layer = extract_distances(shortest_paths_filtered_layer, distance_layer_path)

    iso_areas_output_path = os.path.join(iso_output_dir, f"{layer_name}_iso_areas_filtered.shp")
    create_iso_areas(distance_layer, distance_interval, iso_areas_output_path)

    # Load the filtered Iso-Areas layer to the QGIS map
    iso_areas_filtered_layer = QgsVectorLayer(iso_areas_output_path, f"{layer_name} Iso Areas Filtered", "ogr")
    QgsProject.instance().addMapLayer(iso_areas_filtered_layer)